### 1. Install and Import Dependencies

In [2]:
!pip install mediapipe opencv-python
!pip install ultralytics
!pip install pybboxes

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: /Library/Frameworks/Python.framework/Versions/3.9/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: /Library/Frameworks/Python.framework/Versions/3.9/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: /Library/Frameworks/Python.framework/Versions/3.9/bin/python3 -m pip install --upgrade pip


In [1]:
import mediapipe as mp
import cv2
import numpy as np
from ultralytics import YOLO
import mediapipe as mp
import os
from ultralytics.utils.plotting import Annotator 
import gesture_sim

### 2. Load YOLO stuff

In [2]:
weights_loc = "/Users/taylor/runs/detect/train14/weights/best.pt"
yolo_model = YOLO(weights_loc)

### 3. Hand poses

In [3]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

In [4]:
padding = 30
img_counter = 0
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
desired_aspect_ratio = 1  # 1:1 ratio
standard_size = (350, 350)

In [25]:
def preprocess_image(img):
    hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    lower_pink = np.array([140, 100, 100])
    upper_pink = np.array([170, 255, 255])
    mask = cv2.inRange(hsv_img, lower_pink, upper_pink)
    result = cv2.bitwise_and(img, img, mask=mask)
    return result

def match_gestures(img1, img2, threshold=120):
    img1_processed = preprocess_image(img1)
    img2_processed = preprocess_image(img2)

    orb = cv2.ORB_create()
    kp1, des1 = orb.detectAndCompute(img1_processed, None)
    kp2, des2 = orb.detectAndCompute(img2_processed, None)

    if des1 is not None and des2 is not None and len(des1) > 0 and len(des2) > 0:
        # Convert descriptors to type np.uint8 if they're not already
        if des1.dtype != np.uint8:
            des1 = des1.astype(np.uint8)
        if des2.dtype != np.uint8:
            des2 = des2.astype(np.uint8)

        # Initialize the BFMatcher and perform matching
        bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
        matches = bf.match(des1, des2)
        matches = sorted(matches, key=lambda x: x.distance)

        # Check if the number of matches exceeds the threshold
        if len(matches) > threshold:
            print("The gestures are similar.")
            return True
        else:
            print("The gestures are not similar.")
            return False
    else:
        # Handle the case where one or both sets of descriptors are None or empty
        print("One or both sets of descriptors are missing or empty.")
        return False


In [34]:
cap = cv2.VideoCapture(0) 

with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break  

        #image loading
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image = cv2.flip(image, 1)
        results = hands.process(image)
        canvas_for_yolo = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        h, w, c = frame.shape

        #for drawing
        canvas = np.zeros_like(image)

        #draw results on black canvas
        if results.multi_hand_landmarks:
            for handLMs in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    canvas, handLMs, mp_hands.HAND_CONNECTIONS,
                    mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                    mp_drawing.DrawingSpec(color=(250, 44, 250), thickness=2, circle_radius=2)
                )

        width = canvas_for_yolo.shape[1]

        #perform YOLO predictions
        yolo_results = yolo_model.predict(frame)
        for r in yolo_results:
            #annotate boxes
            annotator = Annotator(canvas)
            boxes = r.boxes
            for box in boxes:
                b = box.xyxy[0]
                mirrored_x_min = width - b[2]
                mirrored_x_max = width - b[0]
                mirrored_box = [mirrored_x_min-padding, b[1]-padding, mirrored_x_max+padding, b[3]+padding]
                c = box.cls
                #annotator.box_label(mirrored_box, "hand")
                
                #create cropped canvas for saving
                save_me = canvas[max(int(mirrored_box[1]), 0):min(int(mirrored_box[3]), canvas.shape[0]),
                               max(int(mirrored_box[0]), 0):min(int(mirrored_box[2]), canvas.shape[1])]


                h, w = save_me.shape[:2]
                current_aspect_ratio = w / h
        
                # Calculate padding
                if current_aspect_ratio < desired_aspect_ratio:
                    # Pad sides
                    new_width = int(desired_aspect_ratio * h)
                    pad_width = (new_width - w) // 2
                    padded_image = cv2.copyMakeBorder(save_me, 0, 0, pad_width, pad_width, cv2.BORDER_CONSTANT, value=[0, 0, 0])
                else:
                    # Pad top and bottom
                    new_height = int(w / desired_aspect_ratio)
                    pad_height = (new_height - h) // 2
                    padded_image = cv2.copyMakeBorder(save_me, pad_height, pad_height, 0, 0, cv2.BORDER_CONSTANT, value=[0, 0, 0])
        
                # Resize image to standard size
                resized_image = cv2.resize(padded_image, standard_size, interpolation=cv2.INTER_AREA)
                base_img  = cv2.imread("base_gestures/stop_base.png")
                start = match_gestures(base_img, resized_image)
                
                if start:
                    cv2.putText(canvas, "stop", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                if cv2.waitKey(10) & 0xFF == ord('s'):
                    img_name = f"cropped_hand_{img_counter}.png"
                    cv2.imwrite("saved_imgs/" + img_name, resized_image)
                    print(f"{img_name} saved.")
                    img_counter += 1

        #display
        cv2.imshow('Hand Skeleton', canvas)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


0: 384x640 (no detections), 193.8ms
Speed: 4.8ms preprocess, 193.8ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 108.5ms
Speed: 1.5ms preprocess, 108.5ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 myright, 96.8ms
Speed: 1.5ms preprocess, 96.8ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
One or both sets of descriptors are missing or empty.

0: 384x640 (no detections), 95.5ms
Speed: 1.4ms preprocess, 95.5ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 myright, 118.4ms
Speed: 2.0ms preprocess, 118.4ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)
One or both sets of descriptors are missing or empty.

0: 384x640 1 myright, 133.3ms
Speed: 1.9ms preprocess, 133.3ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)
One or both sets of descriptors are missing or empty.

0: 384x640 (no detections), 123.8ms
Speed: 2.3ms 